In [6]:
import pandas as pd

# read the data

ori_data = pd.read_csv('X23241.csv')
ori_data.drop(ori_data.columns.values[0], axis=1, inplace=True)

X_train = pd.read_csv('X23241.csv').values[:,1:]
y_train = pd.read_csv('Y23241.csv').values[:,1].ravel().astype(int)

In [7]:
from sklearn.model_selection import train_test_split

# split the data
X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid sizes:', y_train_sub.shape[0], y_valid.shape[0])

Train/Valid sizes: 18592 4649


In [9]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing

# pay attention to the data leaking, we can not use information on validation/test set
scaler = preprocessing.StandardScaler().fit(X_train_sub[:,0:10]) # only use pca for numerical variables

X_scaled_sub = scaler.transform(X_train_sub[:,0:10]) 
X_scaled_valid = scaler.transform(X_valid[:,0:10]) #only use the information from training set

pca = PCA(n_components=5)
pca.fit(X_scaled_sub)
X_new_train = pca.transform(X_scaled_sub)
X_new_valid = pca.transform(X_scaled_valid)

X_std_train = np.hstack((X_new_train, X_train_sub[:,10:]))
X_std_valid = np.hstack((X_new_valid, X_valid[:,10:]))

X_std_train = pd.DataFrame(data = X_std_train)
X_std_valid = pd.DataFrame(data = X_std_valid)
y_train_sub = pd.DataFrame(data = y_train_sub)
y_valid = pd.DataFrame(data = y_valid)

X_std_train.to_csv("pcaX_train.csv", index = False)
X_std_valid.to_csv("pcaX_test.csv", index = False)
y_train_sub.to_csv("pcay_train.csv", index = False)
y_valid.to_csv("pcay_test.csv", index = False)

print(pca.explained_variance_ratio_) # show explained variance ratio

print(pca.components_) # show principal components

[0.25947817 0.21464207 0.17413273 0.10830627 0.07621763]
[[-0.10954727 -0.49016191  0.0999793  -0.10212911 -0.07577068 -0.12853822
   0.45031782 -0.38568849 -0.59430585  0.01760733]
 [-0.3615169   0.15885653  0.52773015 -0.03019004  0.21636961 -0.38312322
  -0.3614256  -0.35200306  0.02936813 -0.34009302]
 [ 0.35184226 -0.07589178  0.13367154  0.64946178  0.61530359  0.13576619
   0.01691891 -0.12718924 -0.1013502   0.08374127]
 [ 0.05280274  0.06185163  0.3599455  -0.22087802 -0.12937005  0.5194458
  -0.28039329 -0.36773785 -0.01528505  0.5607073 ]
 [-0.60324791 -0.05132937 -0.16169364  0.18466062  0.19451425 -0.29182457
   0.01864307  0.13238947  0.08045564  0.65258552]]
